# Парсинг журнала ИК
Данный раздел больше посвящен тому, чтобы навести порядок со структурой и типизацией данных в этом журнале.

In [1]:
import re
import numpy as np
import pandas as pd
import xlrd
import matplotlib.pyplot as plt

## Сохраняем журнал в pd.DataFrame

In [2]:
file_name = 'ИК Папки 2020г.xls'
sheet_name = 'ИК 2020 Р'

In [3]:
xldf = pd.read_excel(file_name, sheet_name=sheet_name,
                    header=1, nrows=170)

# убираем безымянные стобцы
xldf.drop(list(filter(lambda x: re.search(r'^Unnamed:', x) is not None,
                      xldf.columns)), axis=1, inplace=True)

# убираем пробелы по краям названий стобов
xldf.columns = list(map(lambda x: re.sub(r'\s+', ' ', x.strip()).lower().capitalize(), xldf.columns))
xldf

,№ п/п,Дата,№ договора,Наименование организации,Адрес заказчика,Вид работ,Стоимость работ,Место хранения,Исполнитель,Основание,Срок исполнения,Срок действия договора,№ акта,Число,№ протокола,№ заявки
0,5.0,2020-01-09 00:00:00,03/01-20 ИК,"ГБУЗ СК ""Георгиевская РБ""",г. Георгиевск,ИК,115285,Папка Договора ИК\n,Полищук В.М,Договор,20 р.д.,NaN,15,2020-01-14 00:00:00,06025-06055,359
1,6.0,2020-01-14 00:00:00,06/01-20 ИК,ООО «Стоматолог»,г. Пятигорск,ИК,95600,Папка Договора ИК\n,Косиков А.В.,Договор,20 р.д.,NaN,17,2020-01-14 00:00:00,06307-06331,015
2,7.0,2020-01-20 00:00:00,П 08/01-20 ИК,"ГБУЗ СК ""ЦРБ""",г. Прохладный,ИК,25000,Папка Договора ИК\n,Панков А.Г.,Договор,15 р.д.,NaN,42,2020-01-20 00:00:00,06113-06117,338
3,8.0,2020-01-21 00:00:00,П 09/01-20 ИК,"НУЗ ""ОКБ"" на ст. Минеральные Воды ОАО «РЖД»",г. Минеральные Воды,ИК,24420,Папка Договора ИК\n,Панков А.Г.,Договор,15 р.д.,NaN,43,2020-01-21 00:00:00,06332-06334,035
4,9.0,2019-01-27 00:00:00,К 14/01-20 ИК,ГБУЗ РВФД МЗ РСО-А,г.Владикавказ,ИК,37303,Папка Договора ИК\n,Косиков А.В.,Договор,15 р.д.,Нет расчета,64,2020-01-27 00:00:00,06412-06415,047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,138.0,2020-11-12 00:00:00,147/11-20 ИК,"ООО ""Клиника Сити""",г. Невинномысск,ИК,36820,Папка Договора ИК\n,Полищук В.М,Договор,20 р.д.,NaN,1112,2020-11-12 00:00:00,NaN,без заявки
166,NaN,2020-11-12 00:00:00,148/11-20 ИК,"ООО ""Мой доктор""",г. Армавир,ИК,25200,Папка Договора ИК\n,Полищук В.М,Договор,20 р.д.,NaN,NaN,NaN,Заказчик отказался от выполнения работ,без заявки
167,138.0,2020-11-19 00:00:00,149/11-20ИК,ГБУЗ РК «Симферопольская Поликлиника № 5»,"г. Симферополь, п.г.т. Грэсовский",ИК,55800,Папка Договора ИК\n,Полищук В.М,Договор,20 р.д.,NaN,NaN,NaN,Заказчик отказался от выполнения работ,без заявки
168,139.0,2020-11-20 00:00:00,150/11 - 20ИК,ГБУЗ СК «Арзгирская РБ»,с. Арзгир,ИК,24420,Папка Договора ИК\n,Панков\nКосиков,Договор,20 р.д.,NaN,1125,2020-11-20 00:00:00,NaN,296


## Удалим все лишние пробелы и т.д.

In [4]:
new_dtypes = dict(zip(xldf.columns, ['Int64'] + ['string'] * (xldf.shape[1] - 1)))
old_dtypes = dict(zip(xldf.columns, xldf.dtypes))

xldf = xldf.astype(new_dtypes)
new_dtypes

{'№ п/п': 'Int64',
 'Дата': 'string',
 '№ договора': 'string',
 'Наименование организации': 'string',
 'Адрес заказчика': 'string',
 'Вид работ': 'string',
 'Стоимость работ': 'string',
 'Место хранения': 'string',
 'Исполнитель': 'string',
 'Основание': 'string',
 'Срок исполнения': 'string',
 'Срок действия договора': 'string',
 '№ акта': 'string',
 'Число': 'string',
 '№ протокола': 'string',
 '№ заявки': 'string'}

In [5]:
xldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   № п/п                     92 non-null     Int64 
 1   Дата                      170 non-null    string
 2   № договора                170 non-null    string
 3   Наименование организации  170 non-null    string
 4   Адрес заказчика           170 non-null    string
 5   Вид работ                 170 non-null    string
 6   Стоимость работ           168 non-null    string
 7   Место хранения            170 non-null    string
 8   Исполнитель               168 non-null    string
 9   Основание                 169 non-null    string
 10  Срок исполнения           162 non-null    string
 11  Срок действия договора    28 non-null     string
 12  № акта                    144 non-null    string
 13  Число                     144 non-null    string
 14  № протокола               

In [6]:
def remove_spaces(arg):
    return arg.apply(lambda x: re.sub(r'\s+', ' ', x.strip()) if pd.notnull(x) else x)


xldf[xldf.columns[1::]] = xldf[xldf.columns[1::]].apply(remove_spaces, 1)
xldf

,№ п/п,Дата,№ договора,Наименование организации,Адрес заказчика,Вид работ,Стоимость работ,Место хранения,Исполнитель,Основание,Срок исполнения,Срок действия договора,№ акта,Число,№ протокола,№ заявки
0,5,2020-01-09 00:00:00,03/01-20 ИК,"ГБУЗ СК ""Георгиевская РБ""",г. Георгиевск,ИК,115285,Папка Договора ИК,Полищук В.М,Договор,20 р.д.,<NA>,15,2020-01-14 00:00:00,06025-06055,359
1,6,2020-01-14 00:00:00,06/01-20 ИК,ООО «Стоматолог»,г. Пятигорск,ИК,95600,Папка Договора ИК,Косиков А.В.,Договор,20 р.д.,<NA>,17,2020-01-14 00:00:00,06307-06331,015
2,7,2020-01-20 00:00:00,П 08/01-20 ИК,"ГБУЗ СК ""ЦРБ""",г. Прохладный,ИК,25000,Папка Договора ИК,Панков А.Г.,Договор,15 р.д.,<NA>,42,2020-01-20 00:00:00,06113-06117,338
3,8,2020-01-21 00:00:00,П 09/01-20 ИК,"НУЗ ""ОКБ"" на ст. Минеральные Воды ОАО «РЖД»",г. Минеральные Воды,ИК,24420,Папка Договора ИК,Панков А.Г.,Договор,15 р.д.,<NA>,43,2020-01-21 00:00:00,06332-06334,035
4,9,2019-01-27 00:00:00,К 14/01-20 ИК,ГБУЗ РВФД МЗ РСО-А,г.Владикавказ,ИК,37303,Папка Договора ИК,Косиков А.В.,Договор,15 р.д.,Нет расчета,64,2020-01-27 00:00:00,06412-06415,047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,138,2020-11-12 00:00:00,147/11-20 ИК,"ООО ""Клиника Сити""",г. Невинномысск,ИК,36820,Папка Договора ИК,Полищук В.М,Договор,20 р.д.,<NA>,1112,2020-11-12 00:00:00,<NA>,без заявки
166,<NA>,2020-11-12 00:00:00,148/11-20 ИК,"ООО ""Мой доктор""",г. Армавир,ИК,25200,Папка Договора ИК,Полищук В.М,Договор,20 р.д.,<NA>,<NA>,<NA>,Заказчик отказался от выполнения работ,без заявки
167,138,2020-11-19 00:00:00,149/11-20ИК,ГБУЗ РК «Симферопольская Поликлиника № 5»,"г. Симферополь, п.г.т. Грэсовский",ИК,55800,Папка Договора ИК,Полищук В.М,Договор,20 р.д.,<NA>,<NA>,<NA>,Заказчик отказался от выполнения работ,без заявки
168,139,2020-11-20 00:00:00,150/11 - 20ИК,ГБУЗ СК «Арзгирская РБ»,с. Арзгир,ИК,24420,Папка Договора ИК,Панков Косиков,Договор,20 р.д.,<NA>,1125,2020-11-20 00:00:00,<NA>,296


## Достанем список всех исполнителей

In [7]:
# ' '.join(filter(lambda x: x, re.split(r'\W', 'Hello, World!!!')))
xldf['Исполнитель'] =\
xldf['Исполнитель'].apply(lambda item:
                          ' '.join(filter(lambda x: x,
                                          re.split(r'\W', re.sub(r'\b\w\b', '', item))))
                          if pd.notnull(item) else item)

In [8]:
# список исполнителей
perfomancers = set()
xldf['Исполнитель'].apply(lambda x:
                          perfomancers.update(x.split())
                          if pd.notnull(x) else x)
perfomancers

{'Косиков', 'Панков', 'Полищук', 'Тумасов', 'Шеров'}

## Первая версия типизации

In [9]:
xldf = xldf.astype({
    '№ п/п': 'Int64',
    'Дата': 'datetime64',
    '№ договора': 'string',
    'Наименование организации': 'string',
    'Адрес заказчика': 'string',
    'Вид работ': 'string',
    'Стоимость работ': 'string',
    'Место хранения': 'string',
    'Исполнитель': 'string',
    'Основание': 'string',
    'Срок исполнения': 'string',
    'Срок действия договора': 'string',
    '№ акта': 'string',
    'Число': 'datetime64',
    '№ протокола': 'string',
    '№ заявки': 'string'
})

In [10]:
xldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   № п/п                     92 non-null     Int64         
 1   Дата                      170 non-null    datetime64[ns]
 2   № договора                170 non-null    string        
 3   Наименование организации  170 non-null    string        
 4   Адрес заказчика           170 non-null    string        
 5   Вид работ                 170 non-null    string        
 6   Стоимость работ           168 non-null    string        
 7   Место хранения            170 non-null    string        
 8   Исполнитель               168 non-null    string        
 9   Основание                 169 non-null    string        
 10  Срок исполнения           162 non-null    string        
 11  Срок действия договора    28 non-null     string        
 12  № акта                

In [11]:
xldf.head()

,№ п/п,Дата,№ договора,Наименование организации,Адрес заказчика,Вид работ,Стоимость работ,Место хранения,Исполнитель,Основание,Срок исполнения,Срок действия договора,№ акта,Число,№ протокола,№ заявки
0,5,2020-01-09,03/01-20 ИК,"ГБУЗ СК ""Георгиевская РБ""",г. Георгиевск,ИК,115285,Папка Договора ИК,Полищук,Договор,20 р.д.,<NA>,15,2020-01-14,06025-06055,359
1,6,2020-01-14,06/01-20 ИК,ООО «Стоматолог»,г. Пятигорск,ИК,95600,Папка Договора ИК,Косиков,Договор,20 р.д.,<NA>,17,2020-01-14,06307-06331,015
2,7,2020-01-20,П 08/01-20 ИК,"ГБУЗ СК ""ЦРБ""",г. Прохладный,ИК,25000,Папка Договора ИК,Панков,Договор,15 р.д.,<NA>,42,2020-01-20,06113-06117,338
3,8,2020-01-21,П 09/01-20 ИК,"НУЗ ""ОКБ"" на ст. Минеральные Воды ОАО «РЖД»",г. Минеральные Воды,ИК,24420,Папка Договора ИК,Панков,Договор,15 р.д.,<NA>,43,2020-01-21,06332-06334,035
4,9,2019-01-27,К 14/01-20 ИК,ГБУЗ РВФД МЗ РСО-А,г.Владикавказ,ИК,37303,Папка Договора ИК,Косиков,Договор,15 р.д.,Нет расчета,64,2020-01-27,06412-06415,047


## Что лежит в номере акта

In [12]:
# уберем всякие запятые и т.д.
xldf['№ акта'] = xldf['№ акта'].apply(lambda x: '\n'.join(re.findall(r'\b\w+\b', str(x))))

In [13]:
xldf['Адрес заказчика'] = xldf['Адрес заказчика'].apply(lambda x: re.sub(r'\.\s*', '. ', x))

In [14]:
xldf['№ протокола'].value_counts()

Л-аннулированы                               16
Заказчик отказался от выполнения работ       14
Л- анулирован                                11
00077-00081                                   2
00338 - 00339                                 1
                                             ..
00040-00043                                   1
Заказчик отказался, акт, счет 201 обнулен     1
06370-06378                                   1
06746-06748                                   1
06379-06381                                   1
Name: № протокола, Length: 104, dtype: Int64

In [15]:
xldf['№ заявки'].value_counts()

без заявки    29
46             2
177            2
144            2
182            2
              ..
060            1
101            1
12             1
156            1
136            1
Name: № заявки, Length: 134, dtype: Int64

In [16]:
xldf['Стоимость работ'] = xldf['Стоимость работ'].str.replace(' ', '')

xldf['Стоимость работ'] = xldf['Стоимость работ'].str.replace(',', '.')

xldf['Стоимость работ']

0      115285
1       95600
2       25000
3       24420
4       37303
        ...  
165     36820
166     25200
167     55800
168     24420
169     57563
Name: Стоимость работ, Length: 170, dtype: string

,№ п/п,Дата,№ договора,Наименование организации,Адрес заказчика,Вид работ,Стоимость работ,Место хранения,Исполнитель,Основание,Срок исполнения,Срок действия договора,№ акта,Число,№ протокола,№ заявки
